## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-09-16-53-04 +0000,bbc,Mandelson called Epstein 'best pal' in birthda...,https://www.bbc.com/news/articles/cwy9dwe50leo...
1,2025-09-09-16-51-08 +0000,wapo,"U.S. employers added 911,000 fewer jobs than f...",https://www.washingtonpost.com/business/2025/0...
2,2025-09-09-16-50-40 +0000,wapo,Whether U.S. approved Israeli attack in advanc...,https://www.washingtonpost.com
3,2025-09-09-16-50-40 +0000,wapo,Live updates: Israel’s airstrike targeted seni...,https://www.washingtonpost.com/world/2025/09/0...
4,2025-09-09-16-49-18 +0000,bbc,'Butt out of Army row' - O'Neill tells DUP leader,https://www.bbc.com/news/videos/cre571vwy5wo?a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
216,trump,31
13,new,23
113,nepal,14
146,media,14
2,epstein,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
176,2025-09-09-07-15-58 +0000,wapo,"At least 19 killed, hundreds injured in protes...",https://www.washingtonpost.com/world/2025/09/0...,86
296,2025-09-08-20-12-00 +0000,nyt,Trump Issues Warning to Hamas as Israeli Milit...,https://www.nytimes.com/2025/09/08/world/middl...,83
92,2025-09-09-12-55-20 +0000,wapo,Nepal’s prime minister resigns after deadly pr...,https://www.washingtonpost.com/world/2025/09/0...,79
177,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...,74
150,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,74


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
176,86,2025-09-09-07-15-58 +0000,wapo,"At least 19 killed, hundreds injured in protes...",https://www.washingtonpost.com/world/2025/09/0...
296,83,2025-09-08-20-12-00 +0000,nyt,Trump Issues Warning to Hamas as Israeli Milit...,https://www.nytimes.com/2025/09/08/world/middl...
177,74,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...
3,53,2025-09-09-16-50-40 +0000,wapo,Live updates: Israel’s airstrike targeted seni...,https://www.washingtonpost.com/world/2025/09/0...
245,46,2025-09-08-22-53-00 +0000,nypost,Lachlan Murdoch takes control of media empire ...,https://nypost.com/2025/09/08/media/lachlan-mu...
180,44,2025-09-09-06-41-09 +0000,bbc,Thai court rules ex-PM Thaksin must serve one ...,https://www.bbc.com/news/articles/cly7k2g37g4o...
101,36,2025-09-09-12-15-34 +0000,nypost,Cuomo campaign makes last-ditch effort to brea...,https://nypost.com/2025/09/09/us-news/cuomo-ca...
63,35,2025-09-09-14-49-29 +0000,nypost,Greta Thunberg’s Gaza flotilla was not hit by ...,https://nypost.com/2025/09/09/world-news/greta...
129,35,2025-09-09-10-23-48 +0000,nypost,Eerie bodycam footage shows the Amish mother a...,https://nypost.com/2025/09/09/us-news/ohio-ami...
53,33,2025-09-09-15-00-11 +0000,wapo,"Russia strikes eastern Ukraine village, killin...",https://www.washingtonpost.com/world/2025/09/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
